In [1]:
# setup
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import KFold

In [2]:
!pip install -q pyswarms

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 2.6 MB/s eta 0:00:00


In [3]:
import pyswarms as ps

In [ ]:
!pip install -q geneticalgorithm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from geneticalgorithm import geneticalgorithm as ga

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('drive/MyDrive/Data Sets/concrete.csv')

In [5]:
data = pd.read_csv('drive/MyDrive/Data Sets/AirI.csv',header=None)

In [ ]:
data

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
0,3.4482,0.001314,0.157470,0.688300,0.152920,0.000000,0.00000,0.096603,0.52135,0.382050,...,-2.94590,0.219960,-0.69045,0.240440,-0.258640,1.33870,0.114820,-0.43046,0.82309,1.543500
1,3.3019,0.188490,0.553190,0.258290,0.000031,0.000000,0.00000,0.039008,0.43211,0.527160,...,-0.22680,-0.428570,0.74992,-0.111920,1.583000,0.88565,-0.053227,0.67796,-1.72010,-0.550510
2,2.2894,0.052169,0.438520,0.494030,0.015282,0.000000,0.00000,0.000000,0.00000,0.394840,...,1.15790,0.010118,0.73593,1.356000,0.057821,-1.44140,1.581800,1.25420,1.02540,-1.018900
3,2.6207,0.000000,0.000000,0.004213,0.109340,0.493440,0.39300,0.000000,0.00000,0.578540,...,-0.69767,1.577800,0.54899,-0.780420,1.136700,-0.84240,0.993120,-0.44576,-0.78778,-1.490900
4,2.8439,0.000000,0.000000,0.019504,0.260470,0.549860,0.17016,0.010568,0.30520,0.670460,...,-1.49170,-0.688640,-0.69924,0.360740,-0.685860,-0.76161,-0.354560,0.83093,-0.27100,0.388610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,2.4101,0.301000,0.021904,0.000415,0.000000,0.000000,0.00000,0.000000,0.00000,0.051206,...,-1.56660,-1.097100,2.77450,0.679850,-0.292200,0.13691,-0.439250,-1.56390,0.29082,0.023485
107,3.1072,0.000734,0.138890,0.688740,0.171630,0.000000,0.00000,0.171260,0.54746,0.280340,...,-0.32440,-1.341500,-1.51650,0.870840,1.023700,0.26581,0.174520,-0.29601,1.13600,0.217580
108,2.4101,0.001095,0.151180,0.688720,0.159000,0.000000,0.00000,0.000000,0.00000,0.194760,...,-0.49572,-0.832130,0.52386,0.189400,1.115700,-0.14259,-0.481300,-1.58790,-0.23976,1.717000
109,2.6207,0.117370,0.526200,0.354590,0.001831,0.000000,0.00000,0.039008,0.43211,0.527160,...,-0.87353,0.063740,0.20128,0.090828,0.338310,-1.23960,-1.197600,-0.35008,0.59517,0.803430


In [6]:
y = data.iloc[:,0].values
x = data.iloc[:,1:].values

In [ ]:
x = data.loc[:,'cement':'age'].values
y = data['strength'].values

In [17]:
kf = KFold(n_splits=10,shuffle=True,random_state=123)
scale = StandardScaler()

## Example of Objective Format for PSO

The objective function is an assignment between a choice of hyperparameters and a corresponding prediction error.

In [16]:

# the format of the objective function for Particle Swarm Optimization
def objective_pso(h):
  output = []
  for i in range(h.shape[0]):
    a = h[i,0]
    l = h[i,1]
    model = ElasticNet(alpha=a,l1_ratio=l,fit_intercept=True,max_iter=50000)
    PE = []
    for idxtrain, idxtest in kf.split(x):
      xtrain = x[idxtrain]
      ytrain = y[idxtrain]
      xtrain = scale.fit_transform(xtrain)
      xtest = x[idxtest]
      ytest = y[idxtest]
      xtest = scale.transform(xtest)
      model.fit(xtrain,ytrain)
      PE.append(MSE(ytest,model.predict(xtest)))
    output.append(np.mean(PE))
  return output

In [ ]:
h= np.array([[0.01,0.2],[0.25,0.4],[0.3,0.15]])

In [ ]:
h

array([[0.01, 0.2 ],
       [0.25, 0.4 ],
       [0.3 , 0.15]])

In [ ]:
objective_pso(h)

[110.28963379668589, 117.39571870760817, 121.83676508407957]

In [ ]:
def objective_pso_poly(h):
  output = []
  for i in range(h.shape[0]):
    a = h[i,0]
    l = h[i,1]
    model = ElasticNet(alpha=a,l1_ratio=l,max_iter=50000)
    scale = StandardScaler()
    poly = PolynomialFeatures(degree=2)
    pipe = Pipeline([['scaler',scale],['Poly Feats',poly]])
    PE = []
    for idxtrain, idxtest in kf.split(x):
      xtrain = x[idxtrain]
      ytrain = y[idxtrain]
      xtrain = pipe.fit_transform(xtrain)
      xtest = x[idxtest]
      ytest = y[idxtest]
      xtest = pipe.transform(xtest)
      model.fit(xtrain,ytrain)
      PE.append(MSE(ytest,model.predict(xtest)))
    output.append(np.mean(PE))
  return output

## The Application of PSO

In [25]:
# instantiate the optimizer
max = np.array([2,1])
min = np.array([0.001,0.001])
bounds = (min, max)
options = {'c1': 0.25, 'c2': 0.25, 'w': 0.6}
optimizer = ps.single.GlobalBestPSO(n_particles=20, dimensions=2, options=options, bounds=bounds)
cost, pos = optimizer.optimize(objective_pso, iters=150)

2023-11-08 17:20:23,628 - pyswarms.single.global_best - INFO - Optimize for 150 iters with {'c1': 0.25, 'c2': 0.25, 'w': 0.6}
pyswarms.single.global_best: 100%|██████████|150/150, best_cost=0.258
2023-11-08 17:21:52,155 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.2578170747136299, best pos: [1.00974907 0.00486835]


In [ ]:
# instantiate the optimizer
max = np.array([4,1])
min = np.array([0,0])
bounds = (min, max)
options = {'c1': 0.25, 'c2': 0.25, 'w': 0.5}
optimizer = ps.single.GlobalBestPSO(n_particles=20, dimensions=2, options=options, bounds=bounds)
cost, pos = optimizer.optimize(objective_pso_poly, iters=100)

2022-11-03 12:32:36,684 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.25, 'c2': 0.25, 'w': 0.5}
pyswarms.single.global_best: 100%|██████████|100/100, best_cost=60
2022-11-03 12:40:48,856 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 59.98652278739927, best pos: [0.00115233 0.6364007 ]


In [19]:
# this is the format for the Genetic Algorithm and SKLearn Simmulated Annealing
def objective(h):
  a = h[0]
  l = h[1]
  model = ElasticNet(alpha=a,l1_ratio=l,fit_intercept=True,max_iter=50000)
  PE = []
  scale = StandardScaler()
  #poly = PolynomialFeatures(degree=1)
  #pipe = Pipeline([['Scale',scale],['Polynomial Features',poly]])
  #pipe = Pipeline([['Scale',scale]])
  for idxtrain, idxtest in kf.split(x):
    xtrain = x[idxtrain]
    ytrain = y[idxtrain]
    xtrain = scale.fit_transform(xtrain)
    xtest = x[idxtest]
    ytest = y[idxtest]
    xtest = scale.transform(xtest)
    model.fit(xtrain,ytrain)
    PE.append(MSE(ytest,model.predict(xtest)))
  return np.mean(PE)

In [ ]:
varbound=np.array([[0,2],[0,1]])

model=ga(function=objective,dimension=2,variable_type='real',variable_boundaries=varbound)

model.run()

||||||||__________________________________________ 15.3% GA is running...

KeyboardInterrupt: ignored

In [21]:
from scipy.optimize import dual_annealing

In [22]:
lw = [0.00001,0.00001]
up = [2,1]
ret = dual_annealing(objective, bounds=list(zip(lw, up)))

In [23]:
ret.x

array([0.9918403 , 0.00494002])

In [24]:
ret

 message: ['Maximum number of iteration reached']
 success: True
  status: 0
     fun: 0.25781745898160563
       x: [ 9.918e-01  4.940e-03]
     nit: 1000
    nfev: 4376
    njev: 125
    nhev: 0

In [ ]:
ret.x

array([0.22873521, 0.23874174])

In [ ]:
ret

 message: ['Maximum number of iteration reached']
 success: True
  status: 0
     fun: 0.2531573715690361
       x: [ 2.287e-01  2.387e-01]
     nit: 1000
    nfev: 4307
    njev: 102
    nhev: 0

## Simulated Annealing from Scratch

In [ ]:
h_names = ['n_estimators','max_depth']

In [ ]:
model = RandomForestClassifier()

In [ ]:
def GS_Anneal(model,h_names,h1,h2): # h_names means the names of the hyperparameters
  # Here we have an implementation of Simmulated Annealing in Two Dimensions
  MIN_IND1 = 0
  MAX_IND1 = len(h1)
  MIN_IND2 = 0
  MAX_IND2 = len(h2)
  STEP_SIZE = 12    # Std of gaussian steps
  COOLING = 0.98    # T_{n+1} = T_{n}*COOLING, i.e. geometric cooling
  def cost_function(model):
    #pipe = Pipeline([['Scale',scale],['Model',model]])
    #k_fold = cross_val_score(estimator=model,X=x,y=y,cv=10,scoring='neg_mean_squared_error')
    model.fit(xtrain,ytrain)
    return 1 - model.score(xtest,ytest)


  def random_start():
      i = np.random.randint(MIN_IND1,MAX_IND1-1)
      j = np.random.randint(MIN_IND2,MAX_IND2-1)
      pos = [i,j]
      return pos


  def random_neighbour(pos,T):
      i = pos[0]
      j = pos[1]
      raw_i = np.random.normal()*STEP_SIZE*T
      delta_i = np.floor(raw_i).astype(int)
      raw_j = np.random.normal()*STEP_SIZE*T
      delta_j = np.floor(raw_j).astype(int)
      if (delta_i == 0) & (delta_j == 0):
        delta_i = np.sign(raw_i).astype(int)
        delta_j = np.sign(raw_j).astype(int)
      new_i = min(MAX_IND1, max(MIN_IND1, i+delta_i))
      new_j = min(MAX_IND2, max(MIN_IND2, j+delta_j))
      new_pos = [new_i,new_j]
      return new_pos


  def acceptance_probability(cost, new_cost, temperature):
      if new_cost < cost:
          return 1
      else:
          p = np.exp(-(new_cost - cost) / temperature)
          return p


  def locate_min(a):
      smallest = min(a)
      return smallest, [index for index, element in enumerate(a) if smallest == element]

  def annealing(pos0 = random_start()):
      # Start with a random configuration
      # alpha = random_start()
      i = pos0[0]
      j = pos0[1]
      # All layers will have the same number of nodes (keep the example from getting too complicated)
      model.set_params(**{list(h_names)[0]:h1[i-1],list(h_names)[1]:h2[j-1]})

      # Compute the cost function (in this case, 1-accuracy)
      cost = cost_function(model)

      # Keep lists of results, start with the current
      states1, states2, costs = [h1[i-1]], [h2[j-1]], [cost]
      T = 1
      pos = pos0
      while T > 1e-10:
          new_pos = random_neighbour(pos,T)
          i = new_pos[0]
          j = new_pos[1]
          model.set_params(**{list(h_names)[0]:h1[i-1],list(h_names)[1]:h2[j-1]})
          new_cost = cost_function(model)
          if acceptance_probability(cost, new_cost, T) > np.random.random():
              cost = new_cost
              pos = new_pos
              states1.append(h1[pos[0]])
              states2.append(h2[pos[1]])
              costs.append(cost)
          T = T*COOLING
      return model,np.column_stack([states1,states2]),costs
  m,states,costs = annealing()
  return states, costs